# Loc/Roc modules applied to vitalDB 

In [ ]:
import os

import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import vitaldb

from boostlocroc import extract_loc_roc
from boostlocroc.utils import (
    check_and_rescale_units,
    detrend_and_reset_time,
    truncate_fif,
)
from boostlocroc.vis import plot_spectrogram_debug


## Define your criteria and parameters of inerest

In [ ]:
# Use the includded file to get available data per id
Data_available = pd.read_csv("data/VdB_tables/Output_existing_data.csv")
Data_available.head()

In [ ]:
# Read the relevant parameters from a CSV file into a pandas DataFrame
list_relevant_var = pd.read_csv("data/VdB_tables/Relevant_params.csv")

# Extract Arterial pressure params
list_PA = list(list_relevant_var["Params"])[0:11]

# Extract Drug params
list_Drug = list(list_relevant_var["Params"])[11:16]

# Extract EEG pressure params
list_EEG = list(list_relevant_var["Params"])[29:]

# Display the contents of list_EEG
list_EEG

In [ ]:
# make masks based on desired parameters
mask1 = Data_available[list_PA].any(axis=1)
mask2 = Data_available[["Orchestra/PPF20_CE"]].any(axis=1)  # target propofol !!!!
mask3 = Data_available[list_EEG].any(axis=1)

# Apply the masks to filter the DataFrame
filtered_df = Data_available[mask3 & mask2]

# Get the patient IDs from the filtered DataFrame
patient_ids = filtered_df["id"]

# Print the patient IDs
print("There are", len(patient_ids), "ids satisfying your criteria.")

## Make .fif file for LoC/RoC analysis

In [ ]:
output_file = "data/VdB_fif/"
vital_db_directory = ".../VitalDB/vital_files/"


for i in patient_ids[1:5]:
    EEG1 = vitaldb.vital_recs(
        vital_db_directory + i,
        ["BIS/EEG1_WAV"],
        interval=1 / 128,
        return_timestamp=True,
        return_pandas=True,
    )
    EEG2 = vitaldb.vital_recs(
        vital_db_directory + i,
        ["BIS/EEG2_WAV"],
        interval=1 / 128,
        return_timestamp=True,
        return_pandas=True,
    )

    # Detrend and reset time
    EEG1 = detrend_and_reset_time(
        EEG1, var="BIS/EEG1_WAV", new_name="EEG", trend_wind=300
    )
    EEG2 = detrend_and_reset_time(
        EEG2, var="BIS/EEG2_WAV", new_name="EEG", trend_wind=300
    )

    # Metadata
    dt = np.nanmedian(np.diff(EEG2["Time"]))
    sfreq = 1 / dt  # Sampling frequency in Hz
    ch_names = ["EEG1", "EEG2"]  # List of channel names
    ch_types = ["eeg", "eeg"]  # Assuming all channels are EEG

    # Step 2: Create an Info structure
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

    # Step 3: Create Raw object
    eeg_data = np.array([EEG1["EEG"].values, EEG2["EEG"].values])
    raw = mne.io.RawArray(eeg_data, info)

    # Step 4: Save Raw object to .fif file
    output_fname = f"{i}_eeg.fif"

    fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(15, 5))

    ax[0].plot(EEG2["Time"], EEG2["EEG"].values)
    ax[0].set_ylim([-100, 100])

    # Plot spectrogram
    nperseg = int(1.5 * 1 / dt)
    noverlap = int(nperseg / 3)
    pxx, freqs, bins, im = ax[1].specgram(
        EEG2["EEG"].values,
        Fs=1 / dt,
        NFFT=int(nperseg),
        mode="psd",
        cmap="jet",
        noverlap=int(noverlap),
    )
    im.set_clim(-20, 25)
    ax[1].set_title("EEG spectrogram")
    ax[1].set_ylim([0, 32])
    ax[1].set_ylabel("Frequency (Hz)")

    # ! Save the .fif
    # raw.save(output_file + output_fname, overwrite=True)

## Aply Loc/Roc

### Convert if necessary

In [ ]:
directory = os.path.join(vital_db_directory, "VdB_fif")
files = [f for f in os.listdir(directory) if f.endswith(".fif")]

for file in files[0:5]:
    filename = os.path.join(directory, file)

    # Read the raw data
    raw = mne.io.read_raw_fif(filename, preload=True)

    # Compute the median of the absolute values of the signal
    median_value = np.median(np.abs(raw.get_data()))

    if median_value > 5:
        # Rescale the signal
        raw._data *= 10**-6
        raw.save(filename, overwrite=True)
    else:
        print("The units are likely already in volts. No rescaling needed.")

In [ ]:
loc = []
roc = []
name = []
slope = []

for file in files[0:5]:
    filename = os.path.join(directory, file)

    # Unit check
    check_and_rescale_units(filename, threshold=5, new_filename=filename)

    file_tmp = mne.io.read_raw_fif(filename)

    file_tmp = file_tmp.resample(63)
    file_tmp = truncate_fif(file_tmp)
    duration_sec = file_tmp.n_times / file_tmp.info["sfreq"]
    print("sfreq", file_tmp.info["sfreq"])

    if duration_sec > 5 * 60:
        # Extract sampling frequency and make a time vector
        Fs = file_tmp.info["sfreq"]  # Hz
        time = np.linspace(
            0,
            file_tmp.get_data().shape[1] / Fs,
            file_tmp.get_data().shape[1],
            endpoint=False,
        )
        Fp2 = file_tmp.get_data()[1, :] * 10**6  # 10*6 because .fif is saved in Volts

        # Compute LoC and RoC
        time_loc, time_roc, t_proba, proba, LoC_params, RoC_params = extract_loc_roc(
            file_tmp
        )

        loc.append(time_loc)
        roc.append(time_roc)
        name.append(file)
        slope.append(LoC_params[0])

        # Visualize Spectrogram, EEG, and proba
        plot_spectrogram_debug(
            time_loc,
            time_roc,
            Fp2,
            Fs,
            time,
            t_proba,
            proba,
            LoC_params,
            RoC_params,
            filename,
        )

df_label = pd.DataFrame({"id": name, "LoC": loc, "RoC": roc})

In [ ]:
df_label.head()